In [1]:
import os
import sys
import torch
import argparse
import importlib
import numpy as np
from tqdm import tqdm
import multiprocessing
from pathlib import Path
from torch.optim.lr_scheduler import CosineAnnealingLR

sys.path.append("..")
from data_utils.ModelNetDataLoader import ModelNetDataLoader
from data_utils import provider

BASE_DIR = os.path.dirname(os.path.abspath('__file__'))
ROOT_DIR = BASE_DIR
sys.path.append(os.path.join(ROOT_DIR, 'models'))
sys.path.append(os.path.join(ROOT_DIR, 'log'))

# Functions

In [2]:
def test(model, loader, num_class=40, vote_num=1):
    
    mean_correct = []
    class_acc = np.zeros((num_class,3))
    for j, data in tqdm(enumerate(loader), total=len(loader)):
        points, target = data
        target = target[:, 0]
        points = points.transpose(2, 1)
        points, target = points.cuda(), target.cuda()
        classifier = model.eval()
        vote_pool = torch.zeros(target.size()[0],num_class).cuda()
        for _ in range(vote_num):
            pred, _ = classifier(points.float())
            vote_pool += pred
        pred = vote_pool/vote_num
        pred_choice = pred.data.max(1)[1]
        for cat in np.unique(target.cpu()):
            cat = int(cat)
            classacc = pred_choice[target==cat].eq(target[target==cat].long().data).cpu().sum()
            class_acc[cat,0]+= classacc.item()/float(points[target==cat].size()[0])
            class_acc[cat,1]+=1
        correct = pred_choice.eq(target.long().data).cpu().sum()
        mean_correct.append(correct.item()/float(points.size()[0]))
    class_acc[:,2] =  class_acc[:,0]/ class_acc[:,1]
    class_acc = np.mean(class_acc[:,2])
    instance_acc = np.mean(mean_correct)
    return instance_acc, class_acc


def main(args):
    def log_string(str):
        logger.info(str)
        print(str)

    '''HYPER PARAMETER'''
    os.environ["CUDA_VISIBLE_DEVICES"] = args.gpu
    torch.backends.cudnn.enabled = False
    
    '''DATA LOADING'''
    TEST_DATASET = ModelNetDataLoader(root=args.data_root, 
                                       tasks=args.test_tasks,
                                       labels=args.test_labels,
                                       partition='test',
                                       npoint=args.num_point,      
                                       normal_channel=args.normal)
    testDataLoader = torch.utils.data.DataLoader(TEST_DATASET, batch_size=args.batch_size, shuffle=False, num_workers=2)

    '''MODEL LOADING'''
    num_class = 40
    files = os.listdir(args.model_dir+'/logs')
    for f in files:
        if f.endswith('txt'):
            model_name = f.split('.')[0]

    MODEL = importlib.import_module(model_name)

    classifier = MODEL.get_model(num_class,normal_channel=args.normal).cuda()
    checkpoint = torch.load(str(args.model_dir) + '/checkpoints/best_model.pth')
    classifier.load_state_dict(checkpoint['model_state_dict'])

    with torch.no_grad():
        instance_acc, class_acc = test(classifier.eval(), testDataLoader, vote_num=args.num_votes)
        print('Test Instance Accuracy: %f, Class Accuracy: %f' % (instance_acc, class_acc))

In [4]:
class Args:
    batch_size = 32 
    model ='pointnet_cls'  
    ults = 'pointnet_util' 
    gpu = '0' 
    num_point = 2048
    optimizer = 'Adam'
    log_dir = None
    data_root = 'data/'
    normal = False # wheather to use normal
    num_votes = 1   
    
    # available data: original, jitter, translate, missing_part, sparse, rotation, occlusion
    
    # An example for Strategy 1: training-domain validation
    test_tasks = ['test_jitter.npy']                      # define which data you want to test on
    test_labels = ['test_labels.npy']*len(test_tasks)
    model_dir = 'log/classification/2021-02-16_23-09-pointnet_cls-s1' # model checkpoint
    
    

In [5]:
if __name__ == '__main__':
    args = Args()
    main(args) 

The number of test data: 2468


100%|██████████| 78/78 [00:02<00:00, 34.72it/s]

Test Instance Accuracy: 0.873798, Class Accuracy: 0.836477
